<a href="https://colab.research.google.com/github/pxm7946/big-data-challenge/blob/main/level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-01 05:16:32--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2022-03-01 05:16:34 (1.30 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
spark.sparkContext.addFile("https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz")

In [ ]:
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [ ]:
from pyspark.sql.functions import to_date

In [ ]:
reviews_df=df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

In [ ]:
reviews_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R33UPQQUZQEM8|   27288431|B005T4ND06|     400024643| 2015-08-31|
|R3IKTNQQPD9662|   13722556|B004EPZ070|     685335564| 2015-08-31|
|R3U27V5QMCP27T|   20381037|B005S9EKCW|     922008804| 2015-08-31|
|R2TOH2QKNK4IOC|   24852644|B00FC1ZCB4|     326560548| 2015-08-31|
|R2XQG5NJ59UFMY|   15556113|B002ZG98Z0|     637495038| 2015-08-31|
|R1N1KHBRR4ZTX3|    6132474|B00X8RONBO|     896602391| 2015-08-31|
|R3OM9S0TCBP38K|   48049524|B000CEXFZG|     115883890| 2015-08-31|
|R1W4S949ZRCTBW|    3282516|B00ID8H8EW|     977932459| 2015-08-31|
|R18JL1NNQAZFV2|   51771179|B000TGJ8IU|     840084782| 2015-08-31|
|R1LP6PR06OPYUX|   31816501|B00DPMPTDS|     262144920| 2015-08-31|
| RZKBT035JA0UQ|   16164990|B00X797LUS|     883589001| 2015-08-31|
|R253N5W74SM7N3|   33386989|B00C6MXB42|     734735137| 2015-08

In [ ]:
products_df=df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001DE29SS|Chaplin (15th Ann...|
|B00ZJ2GL6G|Jurassic World 3D...|
|B00T5DM876|Pretty Little Lia...|
|B004VW4V76|         Lebanon, PA|
|B009D4SFEC|Murdoch Mysteries...|
|B000GG4Y5K|Numb3rs - The Com...|
|B00383XZP8|TCM Spotlight: Ch...|
|B00I9MS86O|Breaking Bad: The...|
|B00005K3O4|Killer Klowns Fro...|
|B00BD6KETC|     Psych: Season 7|
|B00WHZZBTQ|Heaven Adores You...|
|B000092Q5C|Remo Williams - T...|
|B004AXPWH4|10 MINUTESOLUTION...|
|B00FEP9PQG|Boardwalk Empire:...|
|B0002F6BJW|Fractured Flicker...|
|B0007LXP7W|The A-Team - Seas...|
|B005ET4NHI|  Where The Boys Are|
|B005EBOSO4|Muscle Energy Tec...|
|B003V5CTKU|Black Label Socie...|
|B000087F6L|      Fahrenheit 451|
+----------+--------------------+
only showing top 20 rows



In [ ]:
customers_df=df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   24423656|             8|
|   24297214|             1|
|   12879980|             3|
|     515450|             2|
|   13313689|             1|
|   15523729|             4|
|    1673863|             1|
|   14552054|             2|
|   45392827|            14|
|   44178035|             1|
|   10522786|            10|
|   44848424|             3|
|   14230926|             1|
|   49243158|           274|
|   49084939|            10|
|   28777148|            41|
|   41836864|             1|
|    5219946|             1|
|   37795150|             1|
|   52081222|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
vine_df= df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R33UPQQUZQEM8|          5|            3|          3|   N|
|R3IKTNQQPD9662|          5|            0|          0|   N|
|R3U27V5QMCP27T|          5|            1|          1|   N|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|
|R3OM9S0TCBP38K|          5|            0|          0|   N|
|R1W4S949ZRCTBW|          5|            0|          0|   N|
|R18JL1NNQAZFV2|          5|            0|          0|   N|
|R1LP6PR06OPYUX|          4|            0|          0|   N|
| RZKBT035JA0UQ|          5|            1|          2|   N|
|R253N5W74SM7N3|          4|            1|          1|   N|
|R2D5IFTFPHD3RN|          4|            1|          1|   N|
|R1CECK3H1URK1G|          5|            

In [ ]:
 # Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://"
config = {"user":"root",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [ ]:
 # Write DataFrame to table
reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
 # Write DataFrame to table
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
 # Write DataFrame to table
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
 # Write DataFrame to table
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)